In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/restaurant-revenue-prediction-dataset/restaurant_data.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import Ridge, Lasso
from scipy.stats import uniform
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

In [3]:
df = pd.read_csv('/kaggle/input/restaurant-revenue-prediction-dataset/restaurant_data.csv')
df.head()

,Name,Location,Cuisine,Rating,Seating Capacity,Average Meal Price,Marketing Budget,Social Media Followers,Chef Experience Years,Number of Reviews,Avg Review Length,Ambience Score,Service Quality Score,Parking Availability,Weekend Reservations,Weekday Reservations,Revenue
0,Restaurant 0,Rural,Japanese,4.0,38,73.98,2224,23406,13,185,161.924906,1.3,7.0,Yes,13,4,638945.52
1,Restaurant 1,Downtown,Mexican,3.2,76,28.11,4416,42741,8,533,148.759717,2.6,3.4,Yes,48,6,490207.83
2,Restaurant 2,Rural,Italian,4.7,48,48.29,2796,37285,18,853,56.849189,5.3,6.7,No,27,14,541368.62
3,Restaurant 3,Rural,Italian,4.4,34,51.55,1167,15214,13,82,205.433265,4.6,2.8,Yes,9,17,404556.80
4,Restaurant 4,Downtown,Japanese,4.9,88,75.98,3639,40171,9,78,241.681584,8.6,2.1,No,37,26,1491046.35


In [4]:
#Coulumns from dataset
df.columns

Index(['Name', 'Location', 'Cuisine', 'Rating', 'Seating Capacity',
       'Average Meal Price', 'Marketing Budget', 'Social Media Followers',
       'Chef Experience Years', 'Number of Reviews', 'Avg Review Length',
       'Ambience Score', 'Service Quality Score', 'Parking Availability',
       'Weekend Reservations', 'Weekday Reservations', 'Revenue'],
      dtype='object')

# **Understanding the Data**

**Name**: Identifier for the restaurant

**Location**: Categorical feature indicating the restaurant's location.

**Cuisine**: Categorical feature indicating the type of cuisine.

**Rating**: Numerical feature for average rating.

**Seating Capacity**: Numerical feature for the number of seats.

**Average Meal Price**: Numerical feature for average meal cost.

**Marketing Budget**: Numerical feature for marketing expenses.

**Social Media Followers**: Numerical feature for the number of followers.

**Chef Experience Years**: Numerical feature for the chef's experience.

**Number of Reviews**: Numerical feature for the total reviews.

**Avg Review Length**: Numerical feature for the average review length.

**Ambience Score**: Numerical feature for the ambience.

**Service Quality Score**: Numerical feature for service quality.

**Parking Availability**: Categorical feature indicating if parking is available.

**Weekend Reservations**: Numerical feature for weekend reservations.

**Weekday Reservations**: Numerical feature for weekday reservations.

**Revenue**: Target variable, numerical feature for total revenue.

In [5]:
df.describe()

,Rating,Seating Capacity,Average Meal Price,Marketing Budget,Social Media Followers,Chef Experience Years,Number of Reviews,Avg Review Length,Ambience Score,Service Quality Score,Weekend Reservations,Weekday Reservations,Revenue
count,8368.000000,8368.000000,8368.000000,8368.000000,8368.000000,8368.000000,8368.000000,8368.000000,8368.000000,8368.000000,8368.000000,8368.000000,8.368000e+03
mean,4.008258,60.212835,47.896659,3218.254900,36190.621773,10.051984,523.010397,174.769974,5.521283,5.508772,29.491754,29.235301,6.560706e+05
std,0.581474,17.399488,14.336767,1824.896053,18630.153330,5.516606,277.215127,71.998060,2.575442,2.586552,20.025415,20.004277,2.674137e+05
min,3.000000,30.000000,25.000000,604.000000,5277.000000,1.000000,50.000000,50.011717,1.000000,1.000000,0.000000,0.000000,1.847085e+05
25%,3.500000,45.000000,35.490000,1889.000000,22592.500000,5.000000,277.000000,113.311102,3.300000,3.200000,13.000000,13.000000,4.546514e+05
50%,4.000000,60.000000,45.535000,2846.500000,32518.500000,10.000000,528.000000,173.910079,5.500000,5.600000,27.000000,26.000000,6.042421e+05
75%,4.500000,75.000000,60.300000,4008.500000,44566.250000,15.000000,764.250000,237.406885,7.800000,7.800000,43.000000,43.000000,8.130942e+05
max,5.000000,90.000000,76.000000,9978.000000,103777.000000,19.000000,999.000000,299.984924,10.000000,10.000000,88.000000,88.000000,1.531868e+06


In [6]:
# Calculate the number of null values for each column
null_counts = df.isnull().sum()

# Calculate the percentage of null values for each column
null_percentages = (null_counts / len(df)) * 100

# Combine the results into a DataFrame for better readability
null_summary = pd.DataFrame({
    'Column': df.columns,
    'Null Count': null_counts,
    'Null Percentage': null_percentages
})

# Display the summary
print(null_summary)

                                        Column  Null Count  Null Percentage
Name                                      Name           0              0.0
Location                              Location           0              0.0
Cuisine                                Cuisine           0              0.0
Rating                                  Rating           0              0.0
Seating Capacity              Seating Capacity           0              0.0
Average Meal Price          Average Meal Price           0              0.0
Marketing Budget              Marketing Budget           0              0.0
Social Media Followers  Social Media Followers           0              0.0
Chef Experience Years    Chef Experience Years           0              0.0
Number of Reviews            Number of Reviews           0              0.0
Avg Review Length            Avg Review Length           0              0.0
Ambience Score                  Ambience Score           0              0.0
Service Qual

In [7]:
#Removing Restaurant Name
df = df.drop("Name", axis=1)

In [8]:
#Converting categorical to Numberic
#Location and cuisine are two categorical coulms in data

unique_location = df['Location'].unique()
unique_cuisines = df['Cuisine'].unique()

In [9]:
unique_location

array(['Rural', 'Downtown', 'Suburban'], dtype=object)

In [10]:
unique_cuisines

array(['Japanese', 'Mexican', 'Italian', 'Indian', 'French', 'American'],
      dtype=object)

In [11]:
# One-hot encode the 'Location' column
df = pd.get_dummies(df, columns=['Location'])

In [12]:
# One-hot encode the 'Cuisine' column
df = pd.get_dummies(df, columns=['Cuisine'])

In [13]:
# Ensure all binary columns are integer type
binary_columns = [col for col in df.columns if 'Location_' in col or 'Cuisine_' in col]
df[binary_columns] = df[binary_columns].astype(int)

In [14]:
df.head()

,Rating,Seating Capacity,Average Meal Price,Marketing Budget,Social Media Followers,Chef Experience Years,Number of Reviews,Avg Review Length,Ambience Score,Service Quality Score,...,Revenue,Location_Downtown,Location_Rural,Location_Suburban,Cuisine_American,Cuisine_French,Cuisine_Indian,Cuisine_Italian,Cuisine_Japanese,Cuisine_Mexican
0,4.0,38,73.98,2224,23406,13,185,161.924906,1.3,7.0,...,638945.52,0,1,0,0,0,0,0,1,0
1,3.2,76,28.11,4416,42741,8,533,148.759717,2.6,3.4,...,490207.83,1,0,0,0,0,0,0,0,1
2,4.7,48,48.29,2796,37285,18,853,56.849189,5.3,6.7,...,541368.62,0,1,0,0,0,0,1,0,0
3,4.4,34,51.55,1167,15214,13,82,205.433265,4.6,2.8,...,404556.80,0,1,0,0,0,0,1,0,0
4,4.9,88,75.98,3639,40171,9,78,241.681584,8.6,2.1,...,1491046.35,1,0,0,0,0,0,0,1,0


In [15]:
for i in df.columns:
    print(i, df[i].dtypes)

Rating float64
Seating Capacity int64
Average Meal Price float64
Marketing Budget int64
Social Media Followers int64
Chef Experience Years int64
Number of Reviews int64
Avg Review Length float64
Ambience Score float64
Service Quality Score float64
Parking Availability object
Weekend Reservations int64
Weekday Reservations int64
Revenue float64
Location_Downtown int64
Location_Rural int64
Location_Suburban int64
Cuisine_American int64
Cuisine_French int64
Cuisine_Indian int64
Cuisine_Italian int64
Cuisine_Japanese int64
Cuisine_Mexican int64


In [16]:
df['Parking Availability'].head()

0    Yes
1    Yes
2     No
3    Yes
4     No
Name: Parking Availability, dtype: object

In [17]:
df['Parking Availability'] = df['Parking Availability'].map({'Yes': 1, 'No': 0})

## Models Building

**Here we first try some models with deafult setting, and choose best perfomring model and do further hyperparameter tuning on it.**

In [18]:
# Define the features (X) and target variable (y)
X = df.drop(columns=['Revenue'])
y = df['Revenue']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")


Training data shape: (6694, 22)
Testing data shape: (1674, 22)


**Linear Regressor**

In [19]:
# Initialize the model
LRegressor = LinearRegression()

# Train the model on the training data
LRegressor.fit(X_train, y_train)

LinearRegression()

In [20]:
y_pred = LRegressor.predict(X_test)

In [21]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

In [22]:
print("Mean Squared Error:", mse)
print("R-squared:", r2)
print("Mean Absolute Error:", mae)

Mean Squared Error: 3193598831.241753
R-squared: 0.9554413593451654
Mean Absolute Error: 43217.24979806238


**Gradient Boost Regressor**

In [23]:
GBRegressor = GradientBoostingRegressor()
GBRegressor.fit(X_train,y_train)

GradientBoostingRegressor()

In [24]:
y_pred2 = GBRegressor.predict(X_test)

In [25]:
mse = mean_squared_error(y_test, y_pred2)
r2 = r2_score(y_test, y_pred2)
mae = mean_absolute_error(y_test, y_pred2)

In [26]:
print("Mean Squared Error:", mse)
print("R-squared:", r2)
print("Mean Absolute Error:", mae)

Mean Squared Error: 98654797.52795123
R-squared: 0.9986235203905638
Mean Absolute Error: 7699.2041144357045


**Decision Tree Regressor**

In [27]:
DTRegressor = DecisionTreeRegressor()
DTRegressor.fit(X_train,y_train)

DecisionTreeRegressor()

In [28]:
y_pred3 = DTRegressor.predict(X_test)

In [29]:
mse = mean_squared_error(y_test, y_pred3)
r2 = r2_score(y_test, y_pred3)
mae = mean_absolute_error(y_test, y_pred3)

In [30]:
print("Mean Squared Error:", mse)
print("R-squared:", r2)
print("Mean Absolute Error:", mae)

Mean Squared Error: 229045645.91253388
R-squared: 0.9968042439989864
Mean Absolute Error: 11606.510340501789
